In [1]:
import pandas as pd
from pulp import *

### Parameters

In [2]:
items = ["A", "B", "C", "D", "E", "F"]
baker = {"A": 50, "B": 0, "C": 45, "D": 35, "E": 25, "F": 0}
oven = {"A": 45, "B": 0, "C": 90, "D": 20, "E": 45, "F": 0}
display = {"A": 4, "B": 1.5, "C": 3, "D": 1, "E": 1, "F": 1}
assist = {"A": 0, "B": 12, "C": 0, "D": 0, "E": 0, "F": 8}
profit  = {"A": 6, "B": 4.4, "C": 7.5, "D": 0.9, "E": 1.2, "F": 2.2}

### Initialize Linear Optimization Problem

In [3]:
# 1. Initialize Class
model = LpProblem('Maximize Bakery Profits', LpMaximize)

# 2. Define Decision Variables
A = LpVariable('A', lowBound = 0, cat = 'Integer')
B = LpVariable('B', lowBound = 0, cat = 'Integer')
C = LpVariable('C', lowBound = 0, cat = 'Integer')
D = LpVariable('D', lowBound = 0, cat = 'Integer')
E = LpVariable('E', lowBound = 0, cat = 'Integer')
F = LpVariable('F', lowBound = 0, cat = 'Integer')
var_dict = {'A': A, 'B': B, 'C': C, 'D': D, 'E': E, 'F': F}

C:\Users\User\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


- *LpMaximize*: objective is to maximize the profit
- *lowBound = 0*: Cannot have a negative value of items
- *cat = 'Integer'*: We sell items in entirety.

In [4]:
# 3. Define Objective Function
model += lpSum([profit[i] * var_dict[i] for i in items])

In [5]:
# 4. Define Constaints
# Bakers
model += lpSum([var_dict[i] * baker[i] for i in items]) <= 24*60
# Oven
model += lpSum([var_dict[i] * oven[i] for i in items]) <= 48*60
# Assistant
model += lpSum([var_dict[i] * assist[i] for i in items]) <= 4*60
# Display
model += lpSum([var_dict[i] * display[i] for i in items]) <= 100

### Solve the model

In [6]:
dict_name = dict(zip(["A", "B", "C", "D", "E", "F"],
                    ["Lemon Cake", "Sandwich", "Chocolate Cake", "Croissant", "Chocolate Eclair", "Panini"]))
model.solve()
for v in model.variables():
    print(dict_name[v.name], " = ", int(v.varValue))
    
print("{:,}\t/{:,}\t minutes of bakers used".format(sum([var_dict[i].varValue*baker[i] for i in items]), 24*60))
print("{:,}\t/{:,}\t minutes of oven used".format(sum([var_dict[i].varValue*oven[i] for i in items]), 48*60))
print("{:,}\t/{:,}\t minutes of assistant used".format(sum([var_dict[i].varValue*assist[i] for i in items]), 4*60))
print("{:,}\t/{:,}\t display slots".format(sum([var_dict[i].varValue*display[i] for i in items]), 200))
print("Profit reached per day: {:,} euros".format(round(sum(var_dict[i].varValue * profit[i] for i in items)), 2))


Lemon Cake  =  0
Sandwich  =  20
Chocolate Cake  =  23
Croissant  =  0
Chocolate Eclair  =  1
Panini  =  0
1,060.0	/1,440	 minutes of bakers used
2,115.0	/2,880	 minutes of oven used
240.0	/240	 minutes of assistant used
100.0	/200	 display slots
Profit reached per day: 262 euros


> This model is based on the original constraints stated in the README markdown file. What if we switch scenarios?

#### What if the bakery hired 4 more assistants?

### Modify assistant constraint

In [7]:
# 1. Initialize Class
model = LpProblem('Maximize Bakery Profits', LpMaximize)

# 2. Define Decision Variables
A = LpVariable('A', lowBound = 0, cat = 'Integer')
B = LpVariable('B', lowBound = 0, cat = 'Integer')
C = LpVariable('C', lowBound = 0, cat = 'Integer')
D = LpVariable('D', lowBound = 0, cat = 'Integer')
E = LpVariable('E', lowBound = 0, cat = 'Integer')
F = LpVariable('F', lowBound = 0, cat = 'Integer')
var_dict = {'A': A, 'B': B, 'C': C, 'D': D, 'E': E, 'F': F}

# 3. Define Objective Function
model += lpSum([profit[i] * var_dict[i] for i in items])

# 4. Define Constaints
# Bakers
model += lpSum([var_dict[i] * baker[i] for i in items]) <= 24*60
# Oven
model += lpSum([var_dict[i] * oven[i] for i in items]) <= 48*60
# Assistant
model += lpSum([var_dict[i] * assist[i] for i in items]) <= 4*60*5
# Display
model += lpSum([var_dict[i] * display[i] for i in items]) <= 100

# 5. Solve the model
dict_name = dict(zip(["A", "B", "C", "D", "E", "F"],
                    ["Lemon Cake", "Sandwich", "Chocolate Cake", "Croissant", "Chocolate Eclair", "Panini"]))
model.solve()
for v in model.variables():
    print(dict_name[v.name], " = ", int(v.varValue))
    
print("{:,}\t/{:,}\t minutes of bakers used".format(sum([var_dict[i].varValue*baker[i] for i in items]), 24*60))
print("{:,}\t/{:,}\t minutes of oven used".format(sum([var_dict[i].varValue*oven[i] for i in items]), 48*60))
print("{:,}\t/{:,}\t minutes of assistant used".format(sum([var_dict[i].varValue*assist[i] for i in items]), 4*60*5))
print("{:,}\t/{:,}\t display slots".format(sum([var_dict[i].varValue*display[i] for i in items]), 200))
print("Profit reached per day: {:,} euros".format(round(sum(var_dict[i].varValue * profit[i] for i in items)), 2))


Lemon Cake  =  0
Sandwich  =  66
Chocolate Cake  =  0
Croissant  =  0
Chocolate Eclair  =  0
Panini  =  1
0.0	/1,440	 minutes of bakers used
0.0	/2,880	 minutes of oven used
800.0	/1,200	 minutes of assistant used
100.0	/200	 display slots
Profit reached per day: 293 euros


> It seems like this bakery doesn't have to sell lemon cakes and croissants to stay profitable. But the bakery cannot stop selling them just because so. Therefore, let's add some constraints of minimum quantity for each item to guarantee that customers who love lemon cakes and croissants can still enjoy their favourite product. 

In [8]:
# 1. Initialize Class
model = LpProblem('Maximize Bakery Profits', LpMaximize)

# 2. Define Decision Variables
A = LpVariable('A', lowBound = 0, cat = 'Integer')
B = LpVariable('B', lowBound = 0, cat = 'Integer')
C = LpVariable('C', lowBound = 0, cat = 'Integer')
D = LpVariable('D', lowBound = 0, cat = 'Integer')
E = LpVariable('E', lowBound = 0, cat = 'Integer')
F = LpVariable('F', lowBound = 0, cat = 'Integer')
var_dict = {'A': A, 'B': B, 'C': C, 'D': D, 'E': E, 'F': F}

# 3. Define Objective Function
model += lpSum([profit[i] * var_dict[i] for i in items])

# 4. Define Constaints
# Bakers
model += lpSum([var_dict[i] * baker[i] for i in items]) <= 24*60
# Oven
model += lpSum([var_dict[i] * oven[i] for i in items]) <= 48*60
# Assistant
model += lpSum([var_dict[i] * assist[i] for i in items]) <= 4*60*1
# Display
model += lpSum([var_dict[i] * display[i] for i in items]) <= 100
# Minimum number of Lemon Cake
model += lpSum([var_dict['A']]) == 5
# Minimum number of Croissant
model += lpSum([var_dict['D']]) == 5

# 5. Solve the model
dict_name = dict(zip(["A", "B", "C", "D", "E", "F"],
                    ["Lemon Cake", "Sandwich", "Chocolate Cake", "Croissant", "Chocolate Eclair", "Panini"]))
model.solve()
for v in model.variables():
    print(dict_name[v.name], " = ", int(v.varValue))
    
print("{:,}\t/{:,}\t minutes of bakers used".format(sum([var_dict[i].varValue*baker[i] for i in items]), 24*60))
print("{:,}\t/{:,}\t minutes of oven used".format(sum([var_dict[i].varValue*oven[i] for i in items]), 48*60))
print("{:,}\t/{:,}\t minutes of assistant used".format(sum([var_dict[i].varValue*assist[i] for i in items]), 4*60))
print("{:,}\t/{:,}\t display slots".format(sum([var_dict[i].varValue*display[i] for i in items]), 200))
print("Profit reached per day: {:,} euros".format(round(sum(var_dict[i].varValue * profit[i] for i in items)), 2))


Lemon Cake  =  5
Sandwich  =  20
Chocolate Cake  =  15
Croissant  =  5
Chocolate Eclair  =  0
Panini  =  0
1,100.0	/1,440	 minutes of bakers used
1,675.0	/2,880	 minutes of oven used
240.0	/240	 minutes of assistant used
100.0	/200	 display slots
Profit reached per day: 235 euros


> By simply adding a few lines of code, we can modify this model to fit the bakery owner's needs. 